# Instrukcja - Transformacja Hougha

### Cel:
- zapoznanie z transformacją Hougha dla pojedynczego punktu,
- kilku punktów, prostych figur
- wykorzystanie transformacji Hougha do detekcji linii prostych na rzeczywistym obrazie
- transformacja Hougha w przestrzeni ab

### Transformacja Hough'a

Transformacja Hougha dla prostych jest metodą detekcji współliniowych punktów. Każda prosta może być jednoznacznie przedstawiona za pomocą dwóch parametrów. Przestrzeń tych parametrów to przestrzeń Hougha. Najczęściej wykorzystywanymi parametrami w tej metodzie są współczynniki ρ,θ

opisujące równanie prostej w postaci normalnej:

ρ=x⋅cos(θ)+y⋅sin(θ)

gdzie: ρ - promień wodzący, θ - kąt pomiędzy ρ a osią OX.

Własności transformacji Hougha:
- prostej w przestrzeni kartezjańskiej odpowiada punkt w przestrzeni Hougha
- pękowi prostych przechdzących przez punkt w przestrzeni kartezjańskiej odpowiada krzywa sinusoidalna w przestrzeni Hougha
- punkty leżące na tej samej prostej (w przestrzeni kartezjańskiej) korespondują z sinusoidami przechodzącymi przez wspólny punkt w przestrzeni Hougha.

Metoda wyliczania transformacji Hougha składa się z następujących kroków:
- przez każdy badany (różny od zera) punkt obrazu prowadzony jest pęk prostych, przechodzących przez ten punkt
- każda z tych prostych transformowana jest do przestrzeni Hougha i tworzy tam punkt o współrzędnych ρ,θ
- w ten sposób, każdy punkt obrazu pierwotnego (pęk prostych) jest odwzorowany w sinusoidalną krzywą w przestrzeni Hougha

Przestrzeń Hougha jest przestrzenią akumulacyjną tzn. punkty sinusoidalnych krzywych, wygenerowanych dla punktów obrazu pierwotnego dodają się w miejscach, w których krzywe te przecinają się. Powstałe w ten sposób (w przestrzeni Hougha) maksima odpowiadają zbiorom punktów, należących do jednej prostej. Współrzędne ρ,θ
tego maksimum jednoznacznie określają położenie prostej na obrazie pierwotnym.

### Transformacja Hougha dla małej liczby punktów.
   1. Uruchom poniższy kod. W tablicy `im` wskaż jeden punkt, dla którego ma zostać obliczona transformata.

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from skimage.transform import hough_line, hough_line_peaks
import os

if not os.path.exists("kwadraty.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/11_Hough/kwadraty.png --no-check-certificate
if not os.path.exists("lab112.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/11_Hough/lab112.png --no-check-certificate
if not os.path.exists("dom.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/11_Hough/dom.png --no-check-certificate

im = np.zeros((64,64), dtype=np.uint8)

im[18, 31] = 1

fig, ax = plt.subplots()
fig.set_size_inches(4, 4)
ax.imshow(im, 'gray')
ax.axis('off')


3. Wykonaj transformację Hougha obazu im. Wykorzystaj funkcję *hough_line* z modułu _skimage.transform_. Funkcja zwraca: macierz H (przestrzeń Hougha) oraz dwa wektory theta i rho dla kolejnych 
4. Wyświetl przestrzeń Hougha za pomocą funkcji _plt.imshow_ (można też wykorzystać poniższą funkcję *show_hough*). Jak "wygląda" pojedynczy punkt w przestrzeni Hougha?

In [ ]:
def show_hough(h, image, title: str=None):
    # Generating figure 1
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    ax = axes.ravel()

    ax[0].imshow(image, 'gray')
    ax[0].set_title('Input image')
    ax[0].set_axis_off()

    ax[1].imshow(h, 'gray')
    ax[1].set_title('Hough transform')
    ax[1].set_xlabel('Angles (degrees)')
    ax[1].set_ylabel('Distance (pixels)')
    ax[1].axis('image')
    
    if not title is None:
        fig.suptitle(title, weight='bold')

    plt.tight_layout()
    plt.show()    

In [ ]:
H, th, rho = hough_line(im)
show_hough(H, im)

## Wnioski:
- punkt w przestrzeni Hough'a wygląda jak krzywa siunusoidalna

5. Powtórz punkty 1-4, ale tym razem klinkij dwa punkty. Jak zmienia się przestrzeń Hougha?
6. Powtórz punkty 1-4, ale tym razem kliknij kilka punktów starając się aby były współliniowe. Zaobserwuj zmiany w przestrzeni Hougha
7. Poeksperymentuj z różnymi układami punktów

In [ ]:
def do_hough_points(img_size: tuple, xs: np.ndarray, ys: np.ndarray, title: str='Hough transform') -> None:

    # xs and ys correspond to coordinates of the points, so they must share the same shape
    if xs.shape != ys.shape:
        raise ValueError('Wrong point set.')
    
    img = np.zeros(shape=img_size)
    n,m = img.shape

    for i in range(np.max(xs.shape)):
        x, y = xs[i], ys[i]
        if x < n and y < m:
            img[x,y] = 1
        else:
            continue
    
    img_hough = hough_line(img)[0]

    show_hough(img_hough, img, title=title)

In [ ]:
x = np.array([5,10])
y = x.copy()
img_size = (15,15)
do_hough_points(img_size, x, y, title='5. Two points')

In [ ]:
x = np.array([5,10,15])
y = np.array([10,10,10])
img_size = (20,20)
do_hough_points(img_size, x, y, title='6. Three points on one line')

## Wnioski:
- jeśli punkty są współliniowe, to transformacja hougha dla tych punktów przecinać się będzie w punkcie opisującym prostą, na której leżą.

In [ ]:
x = np.random.randint(1,200,size=(400),dtype=int)
y = np.random.randint(1,200,size=(400),dtype=int)
size=(200,200)
do_hough_points(img_size=size, xs=x, ys=y, title='400 randomly generated point in 200x200 image')

### Transformata Hougha dla pojedynczego obiektu

W tym podpunkcie pokazane zostanie praktycznie wykorzystanie transformaty Hougha - do detekcji prostych na sztucznym rysunku.

   1. Wczytaj obraz "kwadraty.png". Wyświetl go.
   2. Wykonaj detekcję krawędzi jedną z metod gradientowych. Ważne aby obraz krawędzi był jak najlepszej jakości - co oznacza cienkie (nawet niekoniecznie ciągłe) krawędzie - dla tego przypadku nie powinno być trudne do uzyskania. Wyświetl obraz po detekcji krawędzi.
   3. Wykonaj transformatę Hougha obrazu krawędziowego. Wykorzystaj funkcję *hough\_line*.
   4. Wyświetl macierz H. Czy widoczna jest taka liczba maksimów jakiej się spodziewamy?

In [ ]:
squares = cv2.imread('kwadraty.png', cv2.IMREAD_GRAYSCALE)
fig, ax = plt.subplots(1,1,figsize=(5,5))
ax.imshow(squares, 'gray')
ax.axis('off')
_ = fig.suptitle('kwadraty.png', weight='bold')

In [ ]:
from skimage.feature import canny as canny
squares_edges = cv2.Canny(squares, 100,200)
H, th, rh = hough_line(squares_edges)
fig, ax = plt.subplots(1,3, figsize=(12,5))
for a in ax:
    a.axis('off')
ax[0].imshow(squares, 'gray'), ax[0].set_title('original squares')
ax[1].imshow(squares_edges, 'gray'), ax[1].set_title('skimage canny squares')
_=ax[2].imshow(H, 'gray'), ax[2].set_title('hough space')

 5. W module skimage.transform dostępna jest funkcja do automatycznej analizy przestrzeni Hougha - wyszukiwania maksimów - *hough\_line\_peaks*. Jako parametry przyjmuje ona wyniki funkcji *hough\_line* (macierz H, theta i rho). Dodatkowo można podać próg powyżej którego punkt uznawany jest za maksimum (_threshold_ - domyslnie jest to połowa maksimum w przestrzeni H) oraz liczbę poszukiwanych maksimów (*num_peaks*). Funkcja zwraca współrzędne maksimów. Wykorzystaj funkcję *hough\_line\_peaks* do znalezienia maksimów odpowiadających krawędziom kwadratów.
 6. Wyświetl macierz H używając konstrukcji:

In [ ]:
max_value, max_theta, max_rho = hough_line_peaks(H, th, rh)
fig, ax = plt.subplots(1,1, figsize=(15,15))
ax.set_aspect('equal')
ax.axis('off')
ax.imshow(H)

r,p = H.shape
max_theta += np.pi/2
max_theta = np.rad2deg(max_theta)
max_rho += r / 2
plt.imshow(H, 'gray')
plt.title('Hough space with maxima denoted by yellow circles')

for v, t, r in zip(max_value, max_theta, max_rho):
    c = plt.Circle((t, r), 5, color='yellow', fill=False)
    ax.add_patch(c)
plt.show()

]Taki zapis pozwoli na dołożenie annotacji (okręgów) w miejscach znalezionych maksimów. Wyrysowanie okręgu w punkcie x, y (o rozmiarze 10, w czerwonym kolorze, bez wypełnienia środka) realizuje wywołanie: 

**circle = plt.Circle((x, y), 10, color='r', fill=False)**

natomiast dołożenie takiego okręgu do obrazu to:

**ax.add_patch(circle)**

Zaznacz maksima na obrazie wykorzystując rezultat funkcji *hough\_line\_peaks* biorąc pod uwagę, że zwraca ona kąty w radianach z przedziału od -pi/2 do pi/2, a rho z przedziału od -r/2 do r/2 gdzie r to pionowy rozmiar przestrzeni Hougha. 

7. Istnieje też możliwość przeprowadzenia transformacji Hougha z użyciem biblioteki OpenCV. W bibliotece znajdują się dwie wersje funkcji wyszukującej linie proste - 'klasyczna' - _HoughLines_ oraz probabilistyczna _HoughLinesP_. Zadna z nich nie zwraca przestrzeni Hougha. Wynikiem działania pierwszej jest lista parametrów prostych (krotki zawierające rho, theta). Druga zwraca krotki 4-ro elementowe ze współrzędnymi końców odcinków wykorzystanych do wylicznia parametrów (czyli znalezienia prostej). 
8. Wyznacz linie obecne na obrazie za pomocą funkcji _HoughLines_. Wykryte linie wyrysuj na obrazie początkowym (UWAGA: wczytanym bez konwersji na graylevel). Do wyświetlania linii wykorzystaj przykładowy kod:

In [ ]:
hough_lines = cv2.HoughLines(squares_edges, 1, np.pi/180, 50)
squares_c = cv2.imread("kwadraty.png")

for h_line in hough_lines:
    rho, theta = h_line[0, 0], h_line[0, 1]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))  
    img_kwadraty_color = cv2.line(squares_c, (x1, y1), (x2, y2), [0, 255, 0], thickness=1)

plt.figure(figsize=(8,8))
plt.imshow(squares_c)
plt.axis('off')
plt.title("hough lines on image")
plt.show()

9. Wyznacz odcinki obecne na obrazie za pomocą funkcji _HoughLinesP_. Wykryte odcinki wyrysuj na obrazie początkowym (UWAGA: wczytanym bez konwersji na graylevel). 

In [ ]:
squares_c = cv2.imread('kwadraty.png')
squares_c_edge = cv2.Canny(squares_c, 100, 200)
h_lines = cv2.HoughLinesP(squares_c_edge, 1, np.pi/180, 10)

for h_line in h_lines:
    x0, y0, x1, y1 = h_line[0]
    squares_c = cv2.line(squares_c, (x0,y0), (x1,y1), (0,255,0), 2)

fig, ax = plt.subplots(1,1,figsize=(5,5))
ax.imshow(squares_c)
ax.axis('off')
_=ax.set_title('squares with egdes')

### Transformata Hougha dla obrazu rzeczywistego.

Bazując na kodzie stworzonym w punkcie B wyszukamy linie na obrazie rzeczywistym.
   1. Wczytaj obraz "lab112.png". Wyświetl go.
   2. Wykorzystując wszystkie poznane techniki przetwarzania obrazów (filtracja, przekształcenia morfologiczne, binaryzację, detekcję krawędzi) wyodrębnij krawędzie samych kwadratów - tak aby były jak najlepszej jakości (cienkie) - jednocześnie eliminując z obrazu zakłócenia.
   3. Wykorzystaj funkcje *hough_line* i *hough_line_peaks* do detekcji linii na obrazie, a następnie np. wykorzystując kod z punktu 8 poprzedniego ustępu wyrysuj na oryginalnym obrazie znalezione linie.

In [ ]:
lab112 = cv2.imread('lab112.png', cv2.IMREAD_GRAYSCALE)
lab112_edges = cv2.Canny(cv2.GaussianBlur(lab112, (5,5), 1), 130, 240)

fig, ax = plt.subplots(1,2, figsize=(10,5))
for a in ax:
    a.axis('off')
ax[0].imshow(lab112, 'gray'), ax[0].set_title('lab112.png')
_=ax[1].imshow(lab112_edges, 'gray'), ax[1].set_title('after edge detection')

In [ ]:
_H, _th, _rh = hough_line_peaks(*hough_line(lab112_edges))
lab112_c = cv2.imread('lab112.png')

for t, r in zip(_th, _rh):
    a = np.cos(t)
    b = np.sin(t)
    x0 = a*r
    y0 = b*r
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))  
    lab112_c = cv2.line(lab112_c, (x1, y1), (x2, y2), [0, 255, 0], thickness=2)

plt.figure(figsize=(5,5))
plt.imshow(lab112_c)
plt.axis('off')
plt.title('Lab112 with edges')
_ = plt.show()

4. Wczytaj obraz "dom.png". Wypróbuj działanie transformacji Hougha na tym obrazie z wykorzystaniem funkcji _cv2.HoughLinesP_  (oczywiście po odpowiednich przekształceniach). Postaraj się tak przygotować obraz z krawędziami i dobrać parametry aby wyrysować na oryginalnym obrazie odcinki obejmujące zarysy domu. Weź pod uwage dodatkowe parametry funkcji, takie jak:   minLineLength, maxLineGap.

In [ ]:
house = cv2.imread('dom.png', cv2.IMREAD_GRAYSCALE)
house_edges = cv2.Canny(cv2.GaussianBlur(house, (3,3), 1), 130, 240)

fig, ax = plt.subplots(1, 2, figsize=(10,5))
for a in ax:
    a.axis('off')
ax[0].imshow(house,'gray'), ax[0].set_title('dom.png')
_=ax[1].imshow(house_edges,'gray'), ax[1].set_title('house edges')

In [ ]:
h_lines = cv2.HoughLinesP(house_edges, rho=1, theta=np.pi/180, threshold=0, minLineLength=0, maxLineGap=0)
house_c = cv2.imread('dom.png')

for h_line in h_lines:
    x0, y0, x1, y1 = h_line[0]
    house_c = cv2.line(house_c, (x0,y0), (x1,y1), (0,255,0), 2)

plt.figure(figsize=(5,5))
plt.imshow(house_c)
plt.axis('off')
plt.title('House with edges')
_=plt.show()